In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize

import seaborn as sns

import numpy as np
import pandas as pd

from tqdm import tqdm

import torch


In [2]:
from src.model.setup import image_model_setup
model_name = "facebook/dinov2-base"
dataset_name = "temp_dataset_subsample"

model, image_datasets, _ = image_model_setup(model_name, dataset_name, 1000)
model_analysis, _, _ = image_model_setup(model_name, dataset_name, 1000)
model_analysis.to('mps')

W0130 20:10:05.843000 43745 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'


Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from src.initialization.activity_dataset import OnlineLayerInputDataset
from src.model.CLS_token_probing import ModuleSpecificDecoder
from src.train.middle_decoders import train_module_decoder


In [4]:
i = 10
layer_name = f'dinov2.encoder.layer.{i}'

ds_train = OnlineLayerInputDataset(model, layer_name, image_datasets['train'], device='mps')
ds_validation = OnlineLayerInputDataset(model, layer_name, image_datasets['validation'], device='mps')

probe = ModuleSpecificDecoder()
base_module = model_analysis.module_dict[layer_name]

model_out = train_module_decoder(probe, base_module, ds_train, ds_validation)
model_out.save_pretrained(f'layer{i}_probe')

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Currently logged in as: lrast to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



  | Name        | Type                  | Params | Mode 
--------------------------------------------------------------
0 | decoder     | ModuleSpecificDecoder | 2.1 M  | train
1 | base_module | Dinov2Layer           | 7.1 M  | eval 
--------------------------------------------------------------
2.1 M     Trainable params
7.1 M     Non-trainable params
9.2 M     Total params
36.698    Total estimated model params size (MB)
15        Modules in train mode
18        Modules in eval mode


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


...

Metric val/accuracy improved. New best score: 0.755


Validation: |                                             | 0/? [00:00<?, ?it/s]

Metric val/accuracy improved by 0.028 >= min_delta = 0.0. New best score: 0.782


Validation: |                                             | 0/? [00:00<?, ?it/s]

Metric val/accuracy improved by 0.009 >= min_delta = 0.0. New best score: 0.791
`Trainer.fit` stopped: `max_epochs=3` reached.


In [5]:
for i in [8, 9, 11]:
    layer_name = f'dinov2.encoder.layer.{i}'
    
    ds_train = OnlineLayerInputDataset(model, layer_name, image_datasets['train'], device='mps')
    ds_validation = OnlineLayerInputDataset(model, layer_name, image_datasets['validation'], device='mps')
    
    probe = ModuleSpecificDecoder.from_pretrained('layer10_probe')
    base_module = model_analysis.module_dict[layer_name]
    
    model_out = train_module_decoder(probe, base_module, ds_train, ds_validation)
    model_out.save_pretrained(f'layer{i}_probe')

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory ./middle_decoders/epli6v8x/checkpoints exists and is not empty.

  | Name        | Type                  | Params | Mode 
--------------------------------------------------------------
0 | decoder     | ModuleSpecificDecoder | 2.1 M  | train
1 | base_module | Dinov2Layer           | 7.1 M  | eval 
--------------------------------------------------------------
2.1 M     Trainable params
7.1 M     Non-trainable params
9.2 M     Total params
36.698   

Loading weights from local directory


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Metric val/accuracy improved. New best score: 0.749


Validation: |                                             | 0/? [00:00<?, ?it/s]

Metric val/accuracy improved by 0.032 >= min_delta = 0.0. New best score: 0.781


Validation: |                                             | 0/? [00:00<?, ?it/s]

Metric val/accuracy improved by 0.014 >= min_delta = 0.0. New best score: 0.795
`Trainer.fit` stopped: `max_epochs=3` reached.


Overall impressions on the validation accuracy
1. The accuracies are quite impressive. Substantially better than the shuffled accuracies previously. This includes the final layer: progressive read-outs don't appear to ever be necessary.
2. Still slight improvements on epoch 3, could certainly train longer for even better performance.